<a href="https://colab.research.google.com/github/yiguantilburg/school/blob/main/sentiment%20analysi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages

!pip install langdetect

# Download spacy model


# Import libraries used

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy

import re
import nltk
from langdetect import detect
from langdetect import DetectorFactory
from langdetect import detect

 

import warnings
warnings.filterwarnings("ignore")


     |████████████████████████████████| 981 kB 3.6 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=8ace80704aa353f7c17547921a98f63e0116b5b2765d1b78fa840668f1904337
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/My Drive/thesis/airbnb French/")
!ls

Mounted at /content/drive
 cleaned_data_france.csv
 DataEnglish.csv
 DataEnglishFrench.csv
 DataEnglishFrench.xlsx
 DataEnglish.xlsx
 english_clean.pickle
 englishFrench_clean.pickle
 englishFrench_with_location.csv
'English Sentiment.csv'
 english_with_location.csv
 french_eng.csv
'FrenchEnglish Sentiment.csv'
 keep_pun.csv
 Keep_pun_patternNbert.csv
 Keep_pun_patternNbertEnglish.csv
'Keep_pun_patternNbertEnglishGroupby .csv'
 Keep_pun_patternNbertGroupby.csv
 keep_pun.xlsx
 Linear_grid.csv
 listings.csv
 new_listings.csv
 new_listings_english.csv
 new_listings_English_keep_pun.hcsv
 new_listings_English_keep_pun.pickle
 new_listings_english.pickle
 new_listings_French_English_keep_pun.csv
 new_listings_French_English_keep_pun.pickle
 new_listings.pickle
 patternNbert
 patternNbert.csv
 patternNbertEnglish.csv
'pattern sentiment.csv'
 reviews.csv
 test_300comments
 test_300comments.csv
 test_300comments.xlsx
 x_EnglishBert.csv
 x_EnglishBert.pickle
 x_EnglishPattern.csv
 x_EnglishPatt

In [ ]:
import pandas as pd

In [ ]:
with open('reviews.csv') as f:
    reviews_data = pd.read_csv(f)
f.close()
reviews_data.head(10)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,28925,192302096,2017-09-10,149341873,Berend,"To start with, the communication was really go..."
1,28925,286502445,2018-07-06,195637081,Lucie,La maison de Laurence est un véritable havre d...
2,218103,710536,2011-11-11,610681,Dallas,Luis' bed and breakfast is just fantastic. Hi...
3,218103,1109962,2012-04-12,2005513,Nicolas,Four of us had the pleasure of staying in two ...
4,218103,1117012,2012-04-12,1937156,Rob,What a fantastic host and a brilliant place!!!...
5,218103,1128584,2012-04-14,2001261,He,my friends and i ( 5ppl in total) had pleasant...
6,218103,1215056,2012-05-02,1766812,Etienne,Un grand merci à Luis pour ce super séjour ! L...
7,218103,1724783,2012-07-16,1496459,Joanna,Luis was super great! Fantastic host who was h...
8,218103,1964078,2012-08-12,551438,Robert,I knew I picked the right place when Luis woul...
9,218103,2049982,2012-08-20,1582201,Laurent,Nous avons eu un excellent accueil et un séjou...


In [ ]:
# Drop rows that have no comments
reviews_data = reviews_data[reviews_data['comments'].notnull()]
for col in reviews_data.columns:
    print(col, reviews_data[col].isnull().sum())

listing_id 0
id 0
date 0
reviewer_id 0
reviewer_name 0
comments 0


In [ ]:
# preprocess to lower case those comments before identify the language
reviews_data['comments']=reviews_data['comments'].str.lower()

In [ ]:
# Identify review language
DetectorFactory.seed = 0

def lang(text):
  language = ""
  try:
    language = detect(text)
  except:
    language = 'undefined'
  return language

reviews_data['langs'] = reviews_data['comments'].apply(lambda x: lang(x))
reviews_data.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,langs
0,28925,192302096,2017-09-10,149341873,Berend,"to start with, the communication was really go...",en
1,28925,286502445,2018-07-06,195637081,Lucie,la maison de laurence est un véritable havre d...,fr
2,218103,710536,2011-11-11,610681,Dallas,luis' bed and breakfast is just fantastic. hi...,en
3,218103,1109962,2012-04-12,2005513,Nicolas,four of us had the pleasure of staying in two ...,en
4,218103,1117012,2012-04-12,1937156,Rob,what a fantastic host and a brilliant place!!!...,en


In [ ]:
reviews_data.head()

# Drop columns 'date', 'reviewer_Id', 'reviewer_name'
reviews_data.drop(['date', 'reviewer_id', 'reviewer_name'], axis = 1, inplace=True)
reviews_data.head()

,listing_id,id,comments,langs
0,28925,192302096,"to start with, the communication was really go...",en
1,28925,286502445,la maison de laurence est un véritable havre d...,fr
2,218103,710536,luis' bed and breakfast is just fantastic. hi...,en
3,218103,1109962,four of us had the pleasure of staying in two ...,en
4,218103,1117012,what a fantastic host and a brilliant place!!!...,en


In [ ]:
s = reviews_data.langs
counts = s.value_counts()
percent = s.value_counts(normalize=True)
percent100 = s.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pd.DataFrame({'counts': counts, 'per': percent, 'per100': percent100})

,counts,per,per100
fr,182349,0.722469,72.2%
en,53393,0.211544,21.2%
es,5995,0.023752,2.4%
de,2555,0.010123,1.0%
it,1994,0.007900,0.8%
nl,1071,0.004243,0.4%
ca,908,0.003598,0.4%
ro,594,0.002353,0.2%
pt,584,0.002314,0.2%
ru,433,0.001716,0.2%


In [ ]:
##save the preprocessing step till comments lanaguage identification to csv
reviews_data.to_csv("cleaned_data_france.csv")

In [ ]:
import pandas as pd

with open('cleaned_data_france.csv') as f:
    data_france = pd.read_csv(f)
f.close()
data_france.head(10)

,Unnamed: 0,listing_id,id,comments,langs
0,0,28925,192302096,"to start with, the communication was really go...",en
1,1,28925,286502445,la maison de laurence est un véritable havre d...,fr
2,2,218103,710536,luis' bed and breakfast is just fantastic. hi...,en
3,3,218103,1109962,four of us had the pleasure of staying in two ...,en
4,4,218103,1117012,what a fantastic host and a brilliant place!!!...,en
5,5,218103,1128584,my friends and i ( 5ppl in total) had pleasant...,en
6,6,218103,1215056,un grand merci à luis pour ce super séjour ! l...,fr
7,7,218103,1724783,luis was super great! fantastic host who was h...,en
8,8,218103,1964078,i knew i picked the right place when luis woul...,en
9,9,218103,2049982,nous avons eu un excellent accueil et un séjou...,fr


In [ ]:
test_10_comments=data_france.head(10)

data_france.head(10)

In [ ]:
# see how many uniqe vaue 
data_france.nunique()

Unnamed: 0    252397
listing_id      8015
id            252397
comments      243932
langs             44
dtype: int64

In [ ]:
#takes only french and english dataset
french_eng=data_france[data_france['langs'].isin(['en','fr'])]

In [ ]:
#remove url , normalization remove text with less than 3 words,remove punctuation
french_eng.head()

french_eng["comments"] = french_eng["comments"].str.replace('http\S+|www.\S+', '', case=False)
french_eng["comments"] = french_eng["comments"].str.replace('<br/>', '', case=False)
french_eng['comments'] = french_eng['comments'].str.replace('[^\w\s]', '')
french_eng["comments len"]=french_eng["comments"].map(str).apply(len)

more_than_2=french_eng["comments len"]>2
french_eng=french_eng[more_than_2]
french_eng.drop(['comments len'], axis = 1, inplace=True)

french_eng.head()

,Unnamed: 0,listing_id,id,comments,langs
0,0,28925,192302096,to start with the communication was really goo...,en
1,1,28925,286502445,la maison de laurence est un véritable havre d...,fr
2,2,218103,710536,luis bed and breakfast is just fantastic his ...,en
3,3,218103,1109962,four of us had the pleasure of staying in two ...,en
4,4,218103,1117012,what a fantastic host and a brilliant place\nw...,en


In [ ]:
#display all the content of comments

pd.set_option('display.max_colwidth', -1)
print(french_eng["comments"].head(20))

0     to start with the communication was really good and we rent the room a couple hours before we arrived so very fast service the women who rent the place was really hospitable offered us drinks she went to bed and me and a friend could just stay in the room listen to music next morning there was a breakfast for us and we could just leave whenever we wanted pretty good stay when you just want to sleep somewhere while travelling                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
1     la maison de laurence

In [ ]:
keep_pun.to_csv("keep_pun.csv")



In [ ]:
french_eng.to_csv("french_eng.csv")

In [ ]:
import pandas as pd

In [ ]:
with open('french_eng.csv') as f:
    data = pd.read_csv(f)
f.close()

In [ ]:
dataspellcorrect = data.tail(2000)

In [ ]:
!pip install pyspellchecker
from spellchecker import SpellChecker

spellen = SpellChecker()
spellfr = SpellChecker(language='fr')



def spellcorrect(dataspellcorrect):
    if dataspellcorrect['langs']=="en":
      return spellen.correction (dataspellcorrect["comments"])

    else:
      return spellfr.correction (dataspellcorrect["comments"])


dataspellcorrect["comments cleaned"] = dataspellcorrect.apply(lambda x : spellcorrect(x), axis=1)
dataspellcorrect["cleancomments len"]=dataspellcorrect["comments cleaned"].map(str).apply(len)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
dataspellcorrect

notsame=dataspellcorrect["cleancomments len"]!=dataspellcorrect["comments len"]
dataspellcorrect[notsame]

,Unnamed: 0,Unnamed: 0.1,listing_id,id,comments,langs,comments len,comments cleaned,cleancomments len
233975,250517,250634,51488437,472991520878997591,superbe,fr,8,superbe,7
234762,251353,251470,51975777,473654602100761465,super,fr,6,super,5
234813,251411,251528,51994449,493943448762492804,thank you,en,9,thankyou,8
235298,251928,252046,52489636,500563119858551991,parfait,fr,8,parfait,7
235314,251946,252064,52523626,476624700207741676,parfait,fr,8,parfait,7


In [ ]:
#spellcheck is very heavy calulation.Spotcheck 2000 rows, seems thedataquality is quite good. So no need for this step

In [ ]:
!pip install pattern
from pattern.fr import sentiment as sentiment_fr
from pattern.en import sentiment as sentiment_en




def myfunc(data):
    if data['langs']=="en":
        return sentiment_en(data['comments'])[0]
    else:
        return sentiment_fr(data['comments'])[0]

data['sentiment pattern'] = data.apply(lambda x : myfunc(x), axis=1)

     |████████████████████████████████| 22.2 MB 6.7 MB/s 
     |████████████████████████████████| 87 kB 8.4 MB/s 
     |████████████████████████████████| 81 kB 10.0 MB/s 
     |████████████████████████████████| 5.6 MB 71.3 MB/s 
     |████████████████████████████████| 5.6 MB 59.5 MB/s 
     |████████████████████████████████| 419 kB 76.7 MB/s 
     |████████████████████████████████| 104 kB 77.7 MB/s 
     |████████████████████████████████| 3.6 MB 60.6 MB/s 
  Created wheel for pattern: filename=Pattern-3.6-py3-none-any.whl size=22332721 sha256=d7b2ba2ec51e82da57ad02cafc37ebabdf32d111d0f2938ae8108385eba8f574
  Stored in directory: /root/.cache/pip/wheels/8d/1f/4e/9b67afd2430d55dee90bd57618dd7d899f1323e5852c465682
  Created wheel for mysqlclient: filename=mysqlclient-2.1.0-cp37-cp37m-linux_x86_64.whl size=99968 sha256=28470014136ddd6e9cdeacb54b5895eaec316da456e4840e028b97e1b68b7d20
  Stored in directory: /root/.cache/pip/wheels/97/d4/df/08cd6e1fa4a8691b268ab254bd0fa589827ab5b65638c010b4
 

In [ ]:
print(min(data["sentiment pattern"]))
print(max(data["sentiment pattern"]))

-1.0
1.0


In [ ]:
data.to_csv("pattern sentiment.csv")

In [ ]:
## for bert package
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 1.3 MB/s eta 0:15:02tcmalloc: large alloc 1147494400 bytes == 0x394cc000 @  0x7f35d09aa615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████               | 1055.7 MB 1.2 MB/s eta 0:12:24tcmalloc: large alloc 1434370048 bytes == 0x7db22000 @  0x7f35d09aa615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████▋          | 1336.2 MB 1.2 MB/s eta 0:09:00tcmalloc: large alloc 1792966656 bytes =

In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

     |████████████████████████████████| 4.0 MB 4.4 MB/s 
     |████████████████████████████████| 6.6 MB 71.6 MB/s 
     |████████████████████████████████| 77 kB 9.6 MB/s 
     |████████████████████████████████| 895 kB 75.0 MB/s 
     |████████████████████████████████| 596 kB 77.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
## Instantiate Model
##bert-base-multilingual-uncased-sentiment pretained model has taken from Hugging face


tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

In [ ]:
with open('patternNbert.csv') as f:
    patternNbert= pd.read_csv(f)
f.close()
patternNbert.head(10)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,listing_id,id,comments,langs,comments len,sentiment pattern,sentiment_bert
0,0,0,0,0,28925,192302096,to start with the communication was really good and we rent the room a couple hours before we arrived so very fast service the women who rent the place was really hospitable offered us drinks she went to bed and me and a friend could just stay in the room listen to music next morning there was a breakfast for us and we could just leave whenever we wanted pretty good stay when you just want to sleep somewhere while travelling,en,428,0.351667,1.0
1,1,1,1,1,28925,286502445,la maison de laurence est un véritable havre de paix et de transition unique en son genre comme sa propriétaire et ses locataires si chaleureux,fr,143,0.410000,1.0
2,2,2,2,2,218103,710536,luis bed and breakfast is just fantastic his communication is top notch and was maybe one of the most hospitable hosts i have ever had we changed our plans at the last minute and we were worried it wasnt going to work out because it would inconvenience luis who knew his greatest concern would be making sure that we had a great breakfast the room is very quiet and the blinds make the room pitch black which is amazing if you want to sleep in the gardens are very charming the listing says it is in begles and not in bordeaux but i dont know who draws the lines in the city because this is about as close as you can get it takes only a few minutes on the lightrail just around the corner from luis place but we had a car and it took only a few minutes as well the wifi did not work great but that could definitely have been my computer and i did not ask luis so it was not his fault this place is amazing if you want the real french bb experience stay here,en,966,0.318070,1.0
3,3,3,3,3,218103,1109962,four of us had the pleasure of staying in two rooms over easter weekend 2012 we were very impressed by the accommodation which was impeccably clean and nicely presented the bb is a short walk away from the tram stop which takes you to bordeaux stjean station or downtown in a matter of minutes luis our vivacious host was very friendly and approachable you could tell he was passionate about receiving people at his bb and helping them to fully enjoy their time in bordeaux and the surrounding areas the weather wasnt great so we didnt get to fully enjoy the tropical garden urban oasis but wow what a beautiful and tranquil space clearly a labour of love great work luis,en,673,0.423611,0.5
4,4,4,4,4,218103,1117012,what a fantastic host and a brilliant place\nwe arrived into a little piece of paradise our apartment was amazing huge comfortable bed big living area new very modern kitchen and an amazing bathroom after spending a week in paris we really appreciated the water pressure and modern facilities\nbreakfast is served in a beautiful common room luis had everything laid out so wonderfully bespoke jams rich french butter and amazing farm style bread\nwould i stay again i think i could live there forever next year we plan to bring customers from our wine bar for a small tour and book out the entire 4 apartments\nthank luis for the place and your time if you dont know what to do in bordeaux luis is a wealth of information and loves to share,en,737,0.287283,1.0
5,5,5,5,5,218103,1128584,my friends and i 5ppl in total had pleasant stay in this tropical garden luis is an amazing host he was very nice and provided excellent service for use during our stay i really highly recommend this bb it was a shame we didnt really have enough time to enjoy luis french style breadfast but really was great\n\nif you are looking for somewhere convenience warm and comfortable place to stay definately here,en,407,0.419524,1.0
6,6,6,6,6,218103,1215056,un grand merci à luis pour ce super séjour les chambres étaient très agréables les petits déjeuners top \n\nde plus luis nous a donné énormément de conseils sur bordeaux et les alentours \n\nnous tenons tous les 4 à le r

In [ ]:
with open('keep_pun.csv') as f:
    keep_pun = pd.read_csv(f)
f.close()
keep_pun.head(10)

,Unnamed: 0,Unnamed: 0.1,listing_id,id,comments,langs,sentiment bert
0,0,0,28925,192302096,"to start with, the communication was really go...",en,5
1,1,1,28925,286502445,la maison de laurence est un véritable havre d...,fr,5
2,2,2,218103,710536,luis' bed and breakfast is just fantastic. hi...,en,5
3,3,3,218103,1109962,four of us had the pleasure of staying in two ...,en,4
4,4,4,218103,1117012,what a fantastic host and a brilliant place!!!...,en,5
5,5,5,218103,1128584,my friends and i ( 5ppl in total) had pleasant...,en,5
6,6,6,218103,1215056,un grand merci à luis pour ce super séjour ! l...,fr,5
7,7,7,218103,1724783,luis was super great! fantastic host who was h...,en,5
8,8,8,218103,1964078,i knew i picked the right place when luis woul...,en,4
9,9,9,218103,2049982,nous avons eu un excellent accueil et un séjou...,fr,5


In [ ]:
print(min(keep_pun['sentiment bert']))
print(max(keep_pun['sentiment bert']))

1
5


In [ ]:
with open('pattern sentiment.csv') as f:
    pattern_sentiment = pd.read_csv(f)
f.close()
pattern_sentiment.head(10)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,listing_id,id,comments,langs,comments len,sentiment pattern
0,0,0,0,28925,192302096,to start with the communication was really goo...,en,428,0.351667
1,1,1,1,28925,286502445,la maison de laurence est un véritable havre d...,fr,143,0.410000
2,2,2,2,218103,710536,luis bed and breakfast is just fantastic his ...,en,966,0.318070
3,3,3,3,218103,1109962,four of us had the pleasure of staying in two ...,en,673,0.423611
4,4,4,4,218103,1117012,what a fantastic host and a brilliant place\nw...,en,737,0.287283
5,5,5,5,218103,1128584,my friends and i 5ppl in total had pleasant s...,en,407,0.419524
6,6,6,6,218103,1215056,un grand merci à luis pour ce super séjour le...,fr,256,0.385556
7,7,7,7,218103,1724783,luis was super great fantastic host who was he...,en,423,0.238159
8,8,8,8,218103,1964078,i knew i picked the right place when luis woul...,en,818,0.417599
9,9,9,9,218103,2049982,nous avons eu un excellent accueil et un séjou...,fr,404,0.276250


In [ ]:
pattern_sentiment['comments'].iloc[0]
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1




sentiment_score(pattern_sentiment['comments'].iloc[1])

pattern_sentiment['sentiment bert'] = pattern_sentiment['comments'].apply(lambda x: sentiment_score(x[:512]))



In [ ]:
pattern_sentiment



def z_score_standardization(series):
    start = -1
    end = 1
    width = end - start
    nor = (series - series.min())/(series.max() - series.min()) * width + start
    return nor
for col in pattern_sentiment["sentiment bert"]:
    pattern_sentiment['sentiment_bert'] = z_score_standardization(pattern_sentiment['sentiment bert'])
pattern_sentiment.head(20)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,listing_id,id,comments,langs,comments len,sentiment pattern,sentiment bert,sentiment_bert
0,0,0,0,28925,192302096,to start with the communication was really goo...,en,428,0.351667,5,1.0
1,1,1,1,28925,286502445,la maison de laurence est un véritable havre d...,fr,143,0.410000,5,1.0
2,2,2,2,218103,710536,luis bed and breakfast is just fantastic his ...,en,966,0.318070,5,1.0
3,3,3,3,218103,1109962,four of us had the pleasure of staying in two ...,en,673,0.423611,4,0.5
4,4,4,4,218103,1117012,what a fantastic host and a brilliant place\nw...,en,737,0.287283,5,1.0
5,5,5,5,218103,1128584,my friends and i 5ppl in total had pleasant s...,en,407,0.419524,5,1.0
6,6,6,6,218103,1215056,un grand merci à luis pour ce super séjour le...,fr,256,0.385556,5,1.0
7,7,7,7,218103,1724783,luis was super great fantastic host who was he...,en,423,0.238159,5,1.0
8,8,8,8,218103,1964078,i knew i picked the right place when luis woul...,en,818,0.417599,4,0.5
9,9,9,9,218103,2049982,nous avons eu un excellent accueil et un séjou...,fr,404,0.276250,5,1.0


In [ ]:
print(min(pattern_sentiment["sentiment_bert"]))
print(max(pattern_sentiment["sentiment_bert"]))

patternNbert=pattern_sentiment.drop(columns=['sentiment bert'])



patternNbert.to_csv("patternNbert.csv")


patternNbert

#take english review dataset for textblob library
patternNbertEnglish=patternNbert[patternNbert['langs'].isin(['en'])]

patternNbertEnglish.to_csv("patternNbertEnglish.csv")
patternNbertEnglish

-1.0
1.0


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,listing_id,id,comments,langs,comments len,sentiment pattern,sentiment_bert
0,0,0,0,28925,192302096,to start with the communication was really goo...,en,428,0.351667,1.0
2,2,2,2,218103,710536,luis bed and breakfast is just fantastic his ...,en,966,0.318070,1.0
3,3,3,3,218103,1109962,four of us had the pleasure of staying in two ...,en,673,0.423611,0.5
4,4,4,4,218103,1117012,what a fantastic host and a brilliant place\nw...,en,737,0.287283,1.0
5,5,5,5,218103,1128584,my friends and i 5ppl in total had pleasant s...,en,407,0.419524,1.0
...,...,...,...,...,...,...,...,...,...,...
235697,235697,252364,252482,53333751,505637921253892245,very good apartment and hospitality,en,35,0.910000,1.0
235703,235703,252371,252489,53338865,511448695574866240,perfect the room is as seen in the photos clea...,en,209,0.395521,1.0
235719,235719,252390,252508,53497088,512066263929335072,apartment is beautiful has a nice terrace and ...,en,201,0.450000,1.0
235723,235723,252394,252512,53588359,511422906876869104,good apartment for 6 people everything was cle...,en,92,0.658889,0.5


In [ ]:
patternNbertEnglishGroupby = patternNbertEnglish.groupby('listing_id')['sentiment pattern','sentiment_bert']. mean()
print(patternNbertEnglishGroupby.head(10))

patternNbertGroupby = patternNbert.groupby('listing_id')['sentiment pattern','sentiment_bert']. mean()

print(patternNbertGroupby.head(10))


patternNbertGroupby.to_csv("FrenchEnglish Sentiment.csv")


patternNbertEnglishGroupby.to_csv("English Sentiment.csv")

            sentiment pattern  sentiment_bert
listing_id                                   
28925                0.351667        1.000000
218103               0.330609        0.863636
222887               0.429922        0.824324
317273               0.414786        0.880952
317658               0.438787        0.864407
333031               0.405436        0.812500
333592               0.347806        0.738095
365993               0.324592        0.450000
408270               0.404861        0.875000
410267               0.391578        1.000000
            sentiment pattern  sentiment_bert
listing_id                                   
28925                0.380833        1.000000
218103               0.390770        0.842105
222887               0.405652        0.829545
317273               0.393784        0.888889
317658               0.399424        0.810606
333031               0.375070        0.899148
333592               0.315081        0.782051
365993               0.330834     

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


In [ ]:
listings = pd.read_csv('listings.csv')
listings

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,28925,https://www.airbnb.com/rooms/28925,20211211051501,2021-12-11,Grande maison centre ville Comme à la campagne,"If you just open the housedoor, you'll feel li...","A fex minutes from the house, you'll find nice...",https://a0.muscache.com/pictures/miso/Hosting-...,124398,https://www.airbnb.com/users/show/124398,...,5.00,5.00,4.50,33063005371B2,f,1,1,0,0,0.04
1,218103,https://www.airbnb.com/rooms/218103,20211211051501,2021-12-11,B&B Apart.Suite / tropical garden,"<b>The space</b><br />The Bambooyard : ""Le cl...",NaN,https://a0.muscache.com/pictures/9ba23c8f-4780...,1127981,https://www.airbnb.com/users/show/1127981,...,4.96,4.57,4.54,NaN,f,4,4,0,0,0.47
2,222887,https://www.airbnb.com/rooms/222887,20211211051501,2021-12-11,"Bordeaux Terrace - spectacular view, fast Wifi",Picture yourself on a 12 metre terrace overloo...,Bordeaux Terrace is ideally situated to visit ...,https://a0.muscache.com/pictures/8f2b210e-3c0e...,1156398,https://www.airbnb.com/users/show/1156398,...,4.95,4.72,4.49,3306300031048,t,4,4,0,0,0.38
3,317273,https://www.airbnb.com/rooms/317273,20211211051501,2021-12-11,"Bordeaux City Gardens - large 1 bd, park adjacent","A spacious one bedroom apartment, (93 sq metre...",Bordeaux's famous Jardin Public is at the end ...,https://a0.muscache.com/pictures/miso/Hosting-...,1156398,https://www.airbnb.com/users/show/1156398,...,4.93,4.90,4.61,33063001366CB,t,4,4,0,0,0.32
4,317658,https://www.airbnb.com/rooms/317658,20211211051501,2021-12-11,"Key to Bordeaux - fairytale view, 2 bd + elevator",The Key to Bordeaux has a view over the ‘fairy...,The Key to Bordeaux apartment is ideally locat...,https://a0.muscache.com/pictures/a5da668b-39dd...,1156398,https://www.airbnb.com/users/show/1156398,...,4.85,4.97,4.65,33063001225CF,t,4,4,0,0,0.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9466,53721341,https://www.airbnb.com/rooms/53721341,20211211051501,2021-12-11,Maison T2 avec Jardin et piscine,"A Bordeaux, Location bail mobilité 1 à 10 mois...",NaN,https://a0.muscache.com/pictures/miso/Hosting-...,3891465,https://www.airbnb.com/users/show/3891465,...,NaN,NaN,NaN,"Available with a mobility lease only (""bail mo...",t,1,1,0,0,NaN
9467,53737825,https://www.airbnb.com/rooms/53737825,20211211051501,2021-12-11,PARC BORDELAIS T3 fonctionnel proche commodités,"A 2 pas du parc bordelais, de la barrière du m...",appartement entre le parc bordelais et la barr...,https://a0.muscache.com/pictures/c1c90fb3-efde...,65635738,https://www.airbnb.com/users/show/65635738,...,NaN,NaN,NaN,NaN,t,1,1,0,0,NaN
9468,53738324,https://www.airbnb.com/rooms/53738324,20211211051501,2021-12-11,★ Zen & Spa ★ Maison Cosy ★ Eysines ★ Netflix ★,* Unique à Eysines *<br /><br />Envie d'un mom...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,187614996,https://www.airbnb.com/users/show/187614996,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
9469,53745002,https://www.airbnb.com/rooms/53745002,20211211051501,2021-12-11,"Cabane au fond du jardin, Bordeaux - Floirac",Studio indépendant au fond du jardin du propri...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,44265187,https://www.airbnb.com/users/show/44265187,...,NaN,NaN,NaN,NaN,t,1,1,0,0,NaN


In [ ]:
new_listings_French_English = listings.merge(patternNbertGroupby,left_on='id',right_on='listing_id')

new_listings_English = listings.merge(patternNbertEnglishGroupby,left_on='id',right_on='listing_id')

In [ ]:
#pickle
new_listings_French_English.to_pickle('new_listings.pickle')
new_listings_English.to_pickle('new_listings_english.pickle')

In [ ]:
new_listings_French_English.to_csv('new_listings.csv')
new_listings_English.to_csv('new_listings_english.csv')

NameError: ignored

In [ ]:
print(new_listings_French_English.head())

NameError: ignored

In [ ]:
#test:dont remove puntiation remove url , normalization remove text with less than 3 words
keep_pun=french_eng.copy()

keep_pun["comments"] = keep_pun["comments"].str.replace('http\S+|www.\S+', '', case=False)
keep_pun["comments"] = keep_pun["comments"].str.replace('<br/>', '', case=False)

keep_pun["comments len"]=keep_pun["comments"].map(str).apply(len)

more_than_2=keep_pun["comments len"]>2
keep_pun=keep_pun[more_than_2]
keep_pun.drop(['comments len'], axis = 1, inplace=True)

keep_pun.head()

,Unnamed: 0,listing_id,id,comments,langs
0,0,28925,192302096,"to start with, the communication was really go...",en
1,1,28925,286502445,la maison de laurence est un véritable havre d...,fr
2,2,218103,710536,luis' bed and breakfast is just fantastic. hi...,en
3,3,218103,1109962,four of us had the pleasure of staying in two ...,en
4,4,218103,1117012,what a fantastic host and a brilliant place!!!...,en


In [ ]:
keep_pun.to_excel("keep_pun.xlsx")
keep_pun.to_csv("keep_pun.csv")

In [ ]:

#test bert if didn't remove puntiation



def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1




sentiment_score(keep_pun['comments'].iloc[1])

keep_pun['sentiment bert'] = keep_pun['comments'].apply(lambda x: sentiment_score(x[:512]))

keep_pun

,Unnamed: 0,listing_id,id,comments,langs,sentiment bert
0,0,28925,192302096,"to start with, the communication was really go...",en,5
1,1,28925,286502445,la maison de laurence est un véritable havre d...,fr,5
2,2,218103,710536,luis' bed and breakfast is just fantastic. hi...,en,5
3,3,218103,1109962,four of us had the pleasure of staying in two ...,en,4
4,4,218103,1117012,what a fantastic host and a brilliant place!!!...,en,5
...,...,...,...,...,...,...
252392,252510,53588359,509167428203453852,très bien stéphane,fr,5
252393,252511,53588359,510696727952730861,"un hôte super, une super prestation je recomma...",fr,5
252394,252512,53588359,511422906876869104,good apartment for 6 people. everything was cl...,en,5
252395,252513,53606150,511378027315175323,"beautiful, confortable and warm hause, close t...",en,5


In [ ]:
#reassign the bert sentiment score 1-5 to -1  to 1
def re_assign(series):
    start = -1
    end = 1
    width = end - start
    nor = (series - series.min())/(series.max() - series.min()) * width + start
    return nor
for col in keep_pun["sentiment bert"]:
    keep_pun['sentiment_bert'] = re_assign(keep_pun['sentiment bert'])
keep_pun.head(20)

,Unnamed: 0,Unnamed: 0.1,listing_id,id,comments,langs,sentiment bert,sentiment_bert
0,0,0,28925,192302096,"to start with, the communication was really go...",en,5,1.0
1,1,1,28925,286502445,la maison de laurence est un véritable havre d...,fr,5,1.0
2,2,2,218103,710536,luis' bed and breakfast is just fantastic. hi...,en,5,1.0
3,3,3,218103,1109962,four of us had the pleasure of staying in two ...,en,4,0.5
4,4,4,218103,1117012,what a fantastic host and a brilliant place!!!...,en,5,1.0
5,5,5,218103,1128584,my friends and i ( 5ppl in total) had pleasant...,en,5,1.0
6,6,6,218103,1215056,un grand merci à luis pour ce super séjour ! l...,fr,5,1.0
7,7,7,218103,1724783,luis was super great! fantastic host who was h...,en,5,1.0
8,8,8,218103,1964078,i knew i picked the right place when luis woul...,en,4,0.5
9,9,9,218103,2049982,nous avons eu un excellent accueil et un séjou...,fr,5,1.0


In [ ]:
!pip install pattern
from pattern.fr import sentiment as sentiment_fr
from pattern.en import sentiment as sentiment_en




def myfunc(data):
    if data['langs']=="en":
        return sentiment_en(data['comments'])[0]
    else:
        return sentiment_fr(data['comments'])[0]

keep_pun['sentiment pattern'] = keep_pun.apply(lambda x : myfunc(x), axis=1)

In [ ]:
print(min(keep_pun["sentiment_bert"]))
print(max(keep_pun["sentiment_bert"]))
print(min(keep_pun["sentiment pattern"]))
print(max(keep_pun["sentiment pattern"]))

Keep_pun_patternNbert=keep_pun.drop(columns=['sentiment bert'])



Keep_pun_patternNbert.to_csv("Keep_pun_patternNbert.csv")


Keep_pun_patternNbert

#take english review dataset 
Keep_pun_patternNbertEnglish=Keep_pun_patternNbert[Keep_pun_patternNbert['langs'].isin(['en'])]

Keep_pun_patternNbertEnglish.to_csv("Keep_pun_patternNbertEnglish.csv")


-1.0
1.0
-1.0
1.0


In [ ]:
Keep_pun_patternNbertEnglishGroupby = Keep_pun_patternNbertEnglish.groupby('listing_id')['sentiment pattern','sentiment_bert']. mean()
print(Keep_pun_patternNbertEnglishGroupby.head(10))

Keep_pun_patternNbertGroupby = Keep_pun_patternNbert.groupby('listing_id')['sentiment pattern','sentiment_bert']. mean()

print(Keep_pun_patternNbertGroupby.head(10))


Keep_pun_patternNbertGroupby.to_csv("Keep_pun_patternNbertGroupby.csv")


Keep_pun_patternNbertEnglishGroupby.to_csv("Keep_pun_patternNbertEnglishGroupby .csv")

            sentiment pattern  sentiment_bert
listing_id                                   
28925                0.380833        1.000000
218103               0.341727        0.863636
222887               0.445263        0.797297
317273               0.425181        0.880952
317658               0.445980        0.864407
333031               0.416428        0.812500
333592               0.353906        0.738095
365993               0.331020        0.500000
408270               0.420702        0.900000
410267               0.405566        1.000000
            sentiment pattern  sentiment_bert
listing_id                                   
28925                0.395417        1.000000
218103               0.393910        0.850877
222887               0.419801        0.806818
317273               0.400093        0.888889
317658               0.407289        0.856061
333031               0.388003        0.903409
333592               0.319390        0.782051
365993               0.336437     

In [ ]:
with open('Keep_pun_patternNbertGroupby.csv') as f:
    Keep_pun_patternNbertGroupby = pd.read_csv(f)
f.close()

with open('Keep_pun_patternNbertEnglishGroupby .csv') as f:
    Keep_pun_patternNbertEnglishGroupby= pd.read_csv(f)
f.close()

In [ ]:
new_listings_French_English_keep_pun = listings.merge(Keep_pun_patternNbertGroupby,left_on='id',right_on='listing_id')

new_listings_English_keep_pun = listings.merge(Keep_pun_patternNbertEnglishGroupby,left_on='id',right_on='listing_id')


#pickle
new_listings_French_English_keep_pun.to_pickle('new_listings_French_English_keep_pun.pickle')
new_listings_English_keep_pun.to_pickle('new_listings_English_keep_pun.pickle')


new_listings_French_English_keep_pun.to_csv('new_listings_French_English_keep_pun.csv')
new_listings_English_keep_pun.to_csv('new_listings_English_keep_pun.csv')